# Modèles génériques : exercices - Correction

In [ ]:
from mip import *

## Exercice 1

Étienne commence sa préparation en vue de courrir un marathon. Pour se préparer, il doit suivre un régime particulier pour assurer ses besoins quotidiens en énergie (2000kcal), protéïnes (55g) et lipides (800mg). Il peut sélectionner parmi les aliments suivants :



| Plats (portion)  | Énergie (kcal) | Protéïnes (g)| Lipides (mg) |
| -------          | -----          | ---          | ------------ |
| Pâtes bolognaise | 250            | 11           | 2            |
| Omelette fromage | 105            |  8           | 48           |
| Salade César     |  80            | 13           | 54           |
| Hamburger        |  72            | 28           |  88          |
| Curry légumes    | 130            |  6           | 320          |
| Couscous royal   | 180            | 18           | 40           |


Il peut manger plusieurs portions d'un même plat. De plus, les portions peuvent se consommer seulement en partie. 

Les portions ont un coût donné dans le tableau suivant :


| Plats (portion)  | Coût (en centimes) | 
| -------          | -----              |
| Pâtes bolognaise | 50                 | 
| Omelette fromage | 80                 | 
| Salade César     | 140                | 
| Hamburger        | 80                 | 
| Curry légumes    | 40                 | 
| Couscous royal   | 70                 | 



### Question 1

Déterminer un régime permettant à Étienne d'assurer ses besoins en énergie, protéïnes et lipides à un coût minimum.

La formulation que vous donnerez devra être générique.


In [ ]:
#############
#  DONNÉES  #
#############


plats = [
    "Pâtes bolognaise",
    "Omelette fromage",
    "Salade César",
    "Hamburger",
    "Curry légumes",
    "Couscous royal"
]

M = [
    [250, 105, 80, 72, 130, 180],
    [ 11,   8, 13, 28,   6,  18], 
    [  2,  48, 54, 88, 320,  40]
]

prix = [50, 80, 140, 80, 40, 70]
quantités = [2000, 55, 800]

In [ ]:
##################
#   Correction   #
##################


def regime(M, prix, quantités):

    # Création d'un modèle. Ce modèle fera l'interface avec le solveur SCIP
    model = Model()
    
    model.verbose = 0 # Supprimer les log du solveur 

    # Nombre de variables
    m = len(M) 
    n = len(M[0])

    I = range(m) # Indices des contraintes
    J = range(n) # Indices des variables
    
    # On s'assure que les dimensions sont correctes (un prix pour chaque aliment et 
    # une quantité pour chaque besoin alimentaire)
    assert n == len(prix) and m == len(quantités)
    
    # Création d'un tableau de n variables continues positives ou nulles.
    x = [model.add_var() for j in J]
    

    # Création de la fonction objective du modèle
    model.objective = minimize(xsum(prix[j] * x[j] for j in J))

    # Ajout d'une contreainte pour chaque besoin alimentaire
    for i in I:
        model += xsum(M[i][j] * x[j] for j in J) >= quantités[i]
   

    #Résolution du problème d'optimisation linéaire
    status = model.optimize()

    # Le problème doit être optimal (réalisable et borné)
    assert status == OptimizationStatus.OPTIMAL

    return model.objective_value, [x[j].x for j in J]


optimum, sol  = regime(M, prix, quantités)

print(f"coût du régime : {round(optimum / 100, 2)} euros")
for j in range(len(sol)):
    if sol[j] >= 0.001:
        print(f"- {plats[j]} : {round(sol[j], 3)} portions")


### Question 2

Étienne se rend compte qu'il ne peut pas manger trop de portions d'un même plat sans être malade. Il a déterminé qu'il pouvait manger au plus :

| Plats            | Nombre de portions maximum | 
| -------          | -----                      |
| Pâtes bolognaise | 3                          | 
| Omelette fromage | 2                          | 
| Salade César     | 2                          | 
| Hamburger        | 3                          | 
| Curry légumes    | 3                          | 
| Couscous royal   | 2                          | 

Prendre en compte ce nombre maximum de portions dans le modèle.

In [ ]:
##################
#   Correction   #
##################



portionsMax = [ 3, 2, 2, 3, 3, 2]

def regime(M, prix, quantités, portionsMax):

    # Création d'un modèle. Ce modèle fera l'interface avec le solveur SCIP
    model = Model()
    
    model.verbose = 0 # Supprimer les log du solveur 

    # Nombre de variables
    m = len(M) 
    n = len(M[0])

    I = range(m) # Indices des contraintes
    J = range(n) # Indices des variables
    
    # On s'assure que les dimensions sont correctes (un prix pour chaque aliment et 
    # une quantité pour chaque besoin alimentaire)
    assert n == len(prix) and m == len(quantités)
    
    # Création d'un tableau de n variables continues positives ou nulles.
    x = [model.add_var(ub = portionsMax[j]) for j in J]
    

    # Création de la fonction objective du modèle
    model.objective = minimize(xsum(prix[j] * x[j] for j in J))

    # Ajout d'une contreainte pour chaque besoin alimentaire
    for i in I:
        model += xsum(M[i][j] * x[j] for j in J) >= quantités[i]
   

    #Résolution du problème d'optimisation linéaire
    status = model.optimize()

    # Le problème doit être optimal (réalisable et borné)
    assert status == OptimizationStatus.OPTIMAL

    return model.objective_value, [x[j].x for j in J]


optimum, sol  = regime(M, prix, quantités, portionsMax)

print(f"coût du régime : {round(optimum / 100, 2)} euros")
for j in range(len(sol)):
    if sol[j] >= 0.001:
        print(f"- {plats[j]} : {round(sol[j], 3)} portions")


## Exercice 2


Une entreprise d'électricité doit fournir de l’électricité pour les villes de Villetaneuse, Épinay et Saint-Denis. La puissance nécessaire d'électricité pour chaque ville est respectivement de 3000, 6000 et 5000 GW. Pour produire cette électricité, l’entreprise dispose de trois centrales électriques C1, C2 et C3. Le coût de production et d’acheminement de l’électricité en fonction des villes et des centrales (en &euro; par GW) est récapitulée dans le tableau suivant.



| Centrales | Villetaneuse | Épinay | Saint-Denis |
| ------- | -----        | ---    | ------------|
| C1      | 80           | 100    | 120         |
| C2      | 20           | 30     | 40          |
| C3      | 40           | 60     | 80          |


De plus, chaque centrale possède une puissance limitée. Cette limite est respectivement de 9000, 6000 et 7000 GW. L’entreprise souhaite déterminer la production d’électricité par centrale ainsi que la répartition de la production en fonction des trois villes de telle manière que le coût soit minimal. 



In [ ]:
#############
#  DONNÉES  #
#############


centrales = ["C1", "C2", "C3"]
puissances_max = [9000, 6000, 7000]

villes = ["Villetaneuse", "Épinay", "Saint-Denis"]
demandes = [3000, 6000, 5000]

coûts = [
    [80, 100, 120],
    [20,  30,  40],
    [40,  60,  80]
]




C = range(len(centrales))
V = range(len(villes))

# Affichage des données
print("Centrales :")
for c in C:
    print(f"- Centrale {centrales[c]}, puissance maximum = {puissances_max[c]} GW")

print("Villes :")
for v in V:
    print(f"- Ville {villes[v]}, demande = {demandes[v]} GW")

print("Coûts :")
for c in C:
    for v in V:
        print(f"- Coût de {centrales[c]} à {villes[v]} = {coûts[c][v]} euros/GW")



### Question 1

Formuler ce problème sous forme générique.

In [ ]:
##################
#   Correction   #
##################


def optimise_centrales(coûts, puissances_max, demandes):

    # Création d'un modèle. 
    model = Model()
    
    model.verbose = 0 # Supprimer les log du solveur 

    # On s'assure que les dimensions sont correctes 
    assert len(coûts) == len(puissances_max) and len(coûts[0]) == len(demandes)
    
    
    C = range(len(puissances_max)) # Ensemble des centrales
    V = range(len(demandes)) # Ensemble des villes

    # x[c][v] correspond à la quantité d'électricité produite et acheminée 
    # de la centrale numéro c à la ville numéro v
    x = [
        [ model.add_var() for v in V] for c in C
    ]

    # Création de la fonction objective du modèle
    model.objective = minimize(xsum(coûts[c][v] * x[c][v] for c in C for v in V))

    # La puissance délivrée par chaque centrale doit être inférieure ou égale à sa puissance max
    for c in C:
        model += xsum(x[c][v] for v in V) <= puissances_max[c]

    for v in V:
        model += xsum(x[c][v] for c in C) >= demandes[v]
    
    #Résolution du problème d'optimisation linéaire
    status = model.optimize()

    # Le problème doit être optimal (réalisable et borné)
    assert status == OptimizationStatus.OPTIMAL

    return model.objective_value, [[x[c][v].x for v in V] for c in C]



optimum, solution  = optimise_centrales(coûts, puissances_max, demandes)
print(f"Coût : {round(optimum, 2)} euros")
   
for c in C:
    for v in V:
        if solution[c][v] >= 0.001:
            print(f"La centrale {centrales[c]} produit {round(solution[c][v], 3)} GW pour la ville {villes[c]}")


### Question 2

Modifier uniquement les données en ajoutant une quatrième centrale de puissance maximum 3000 GW. Cette centrale peut délivrer l'électricité aux villes selon les coûts : 

|Villetaneuse | Épinay | Saint-Denis|
| -----        | ---    | ------------|
|10           | 20    | 15         |

Cela change-t-il la solution ?

In [ ]:
##################
#   Correction   #
##################


coûts_nouvelle_centrale = [10, 20, 15]

coûts.append(coûts_nouvelle_centrale)
puissances_max.append(3000)

optimum, solution  = optimise_centrales(coûts, puissances_max, demandes)
print(f"Coût : {round(optimum, 2)} euros")
   
for c in C:
    for v in V:
        if solution[c][v] >= 0.001:
            print(f"La centrale {centrales[c]} produit {round(solution[c][v], 3)} GW pour la ville {villes[c]}")

# Cela change la solution

## Exercice 3 : Epin&Drink&reg;

Une entreprise d'Épinay souhaite lancer en 2025 une nouvelle boisson nommée **Epin&Drink&reg;**. Le service commercial a réalisé une étude et ses prévisions de vente en hectolitres (hl) pour chaque mois de l'année 2025 sont données dans le tableau suivant :

|Mois | Janvier | Février | Mars | Avril | Mai | Juin | Juillet | Août | Septembre | Octobre | Novembre | Décembre|
|--- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | ---|
| Volumes (hl) | 200 | 180 | 210 | 178 | 247 | 302 | 354 | 398 | 253 | 265 | 264 | 299|


Le coût de fabrication d'un hectolitre d'**Epin&Drink&reg;** varie tous les mois (suivant les équipes et les autres produits fabriqués par cette entreprise). Les estimations de ce coût sont donnés dans le tableau suivant :

| Janvier | Février | Mars | Avril | Mai | Juin | Juillet | Août | Septembre | Octobre | Novembre | Décembre|
|--- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 54 | 63 | 32 | 55 | 96 | 24 | 46 | 94 | 116 | 87 | 75 | 73 |

Afin de satisfaire la demande et d'optimiser ses coûts de fabrication, l'entreprise peut stocker chaque mois une quantité de boisson pour la vendre plus tard (il n'y a aucun problème de date de péremption). Ce stockage coûte 40 &euro;/hectolitre/mois. La boisson fabriquée et vendue le même mois n'a pas besoin d'être stockée.

L'entreprise ayant déjà réalisé un test de production, elle possède 50 hectolitres de boisson au début du mois de janvier. De plus, elle peut au maximum produire 350 hectolitres et stocker 250 hectolitres chaque mois.

En tant qu'informaticien de cette entreprise, vous devez déterminer le plan de production de l'année 2025. Cette production doit permettre chaque mois de vendre le volume **Epin&Drink&reg;** prévu et le coût de cette production doit être minimum.
Quel est le coût total de fabrication/stockage sur l'année ?

In [ ]:
#############
#  DONNÉES  #
#############

#Volume des ventes
demandes = [200, 180, 210, 178, 247, 302, 354, 398, 253, 265, 264, 299]

c_prod = [54, 63, 32, 55, 96, 24, 46, 94, 116, 87, 75, 73]

c_stock = 40

s_deb = 50

prod_max = 350

stock_max = 250

Mois = range(len(demandes)) # ensemble des mois

In [ ]:
##################
#   Correction   #
##################


def epinEtDrink(demandes, c_prod, c_stock, s_deb, prod_max, stock_max):

    Mois = range(len(demandes)) # ensemble des mois
    
    # Création d'un modèle. 
    model = Model()
    
    model.verbose = 0 # Supprimer les log du solveur 

    # p[m] est la quantité de boisson produite durant le mois m
    p = [model.add_var(ub = prod_max) for m in Mois]
    
    
    # s[m] est la quantité de boisson stockée au mois m pour être vendue plus tard (un autre mois)
    s = [model.add_var(ub = stock_max) for m in Mois]
    
    
    model.objective = minimize(xsum(c_prod[m] * p[m] for m in Mois) + c_stock * sum(s[m] for m in Mois))

    for m in Mois:
        if m == 0:
            model += p[m] + s_deb == demandes[m] + s[m]
        else:
            model += p[m] + s[m-1] == demandes[m] + s[m]
        
    #Résolution du problème d'optimisation linéaire
    status = model.optimize()

    # Le problème doit être optimal (réalisable et borné)
    assert status == OptimizationStatus.OPTIMAL

    return model.objective_value, [p[m].x for m in Mois], [s[m].x for m in Mois]

coût, prod, stock =  epinEtDrink(demandes, c_prod, c_stock, s_deb, prod_max, stock_max)



print(f"coût : {round(coût, 2)} euros")
   

    
for m in Mois:
        print(f"Mois {m + 1} : Quantite produite = {prod[m]} et quantité stockée = {stock[m]}")


## Exercice 4 :  (issu du cours de Roland Grappe)

Une entreprise dispose de deux machines pour produire trois boissons. Les rythmes de production doivent être : 
* Epin&Drink&reg; : 42 litres par heure, 
* Epin&Drink&reg;Max&trade; : 49 litres par heure, 
* Villeta'up&trade; : 25 litres par heure.

Pour produire un litre de boisson, chaque machine met le temps suivant, en minutes :

|  -  | Machine 1 | Machine 2 |
|----|-----------|-----------|
| Epin&Drink&reg; | 5 | 6 |
| Epin&Drink&reg;Max&trade; | 3 |5|
| Villeta'up&trade; | 2 |

Les depenses de fabrication sont, par litre produit :

|  -  | Machine 1 | Machine 2 |
|----|-----------|-----------|
| Epin&Drink&reg; | 20 | 22 |
| Epin&Drink&reg;Max&trade; | 8 |12|
| Villeta'up&trade; | 12 |

**Questions :**

1. Formuler le problème qui consiste à répartir les productions sur les deux machines afin de minimiser les dépenses de production.
2. Écrire ce problème sous forme générique.
3. Modifer uniquement les données pour ajouter une troisième machine.

In [ ]:
#############
#  DONNÉES  #
#############

# bigM représente une très grande valeur (assimilable à +inf)
import sys
bigM = sys.maxsize

Boissons = ["Epin&drink", "Epin&drinkMax", "Villeta'up"]
Machines = ["M1", "M2"]

temps = [
    [5, 3,    2],
    [6, 5, bigM]
]

dépenses = [
    [20,  8,   12],
    [22, 12, bigM]
]

productions = [42, 49, 25]

M = range(len(Machines)) # ensemble des machines
B = range(len(Boissons)) # ensemble des boissons

In [ ]:
##################
#   Correction   #
##################


def productionBoisson(dépenses, temps, productions):

    
    M = range(len(dépenses)) # ensemble des machines
    B = range(len(dépenses[0])) # ensemble des boissons
    
    # Création d'un modèle. 
    model = Model()
    
    model.verbose = 0 # Supprimer les log du solveur 

    # x[m][b] est la quantité de boisson (L) de type b produite sur la machine m en une heure
    x = [
        [model.add_var() for b in B] for m in M
    ]
    
        
    model.objective = minimize(xsum(dépenses[m][b] * x[m][b] for m in M for b in B))
    
    for b in B:
        model += xsum(x[m][b] for m in M) >= productions[b]
    
    
    for m in M:
        model += xsum(temps[m][b] * x[m][b] for b in B) <= 60
    
    
    #Résolution du problème d'optimisation linéaire
    status = model.optimize()

    # Le problème doit être optimal (réalisable et borné)
    assert status == OptimizationStatus.OPTIMAL
    
    return model.objective_value, [ [x[m][b].x for b in B] for m in M]


coûts, prod =  productionBoisson(dépenses, temps, productions)

print(f"coût : {round(coûts, 2)} euros")
   

for m in M:
    print(f"{Machines[m]} produit :")
    for b in B:
            if prod[m][b] > 0:
                print(f"   - {round(prod[m][b], 3)}L de {Boissons[b]}")
